In [3]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define transformation (same as used in training)
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Load class names
train_dir = "/kaggle/input/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train"
class_names = sorted(os.listdir(train_dir))

# Load model
num_classes = len(class_names)
model = models.efficientnet_v2_l(pretrained=False)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = nn.DataParallel(model).to(device)

# Load trained weights
model.load_state_dict(torch.load("/kaggle/input/plant-effnet-v1/0.005133384762423619 12.pth"))  # Replace with actual .pth filename
model.eval()

# Inference function
def predict(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
        predicted_class = class_names[predicted.item()]
    
    return predicted_class

# Example usage


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_31/1579001411.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the function

In [6]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Path to test dataset
test_dir = "/kaggle/input/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid"  # Assuming validation set is used for testing

# Create test dataset and loader
test_dataset = ImageFolder(test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Accuracy calculation
correct = 0
total = 0

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Test Accuracy: 99.40%


In [7]:
k="/kaggle/input/new-plant-diseases-dataset/test/test"
print(predict("/kaggle/input/new-plant-diseases-dataset/test/test/TomatoYellowCurlVirus2.JPG"))

Tomato___Tomato_Yellow_Leaf_Curl_Virus
